In [1]:
import os 
import keras 
import numpy as np
import tensorflow as tf

# Data
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

# Model
from keras.models import Sequential, load_model
from keras.layers import Dense

# Callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Metrics
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, classification_report

In [4]:
data = pd.read_csv('smoke_detection_iot.csv', )
# data.drop(['Unnamed'], axis=1)
data.head()

,Unnamed: 0,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.0,0.0,0.0,0.0,0.0,0,0
1,1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.0,0.0,0.0,0.0,0.0,1,0
2,2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.0,0.0,0.0,0.0,0.0,2,0
3,3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.0,0.0,0.0,0.0,0.0,3,0
4,4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.0,0.0,0.0,0.0,0.0,4,0


In [5]:
data.drop(['Unnamed: 0', 'UTC'], axis=1, inplace=True)


In [6]:
y = data.pop('Fire Alarm').to_numpy()
X = data

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
spliter = StratifiedShuffleSplit(n_splits=3, test_size=0.2)
for train_ids, test_ids in spliter.split(X_scaled, y):
    X_train, y_train = X_scaled[train_ids], y[train_ids]
    X_test, y_test = X_scaled[test_ids], y[test_ids]

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)


In [10]:
X_train.shape

(40083, 13)

In [11]:
# Model Architecture
model1 = Sequential([
    Dense(32, activation='relu', kernel_initializer='he_normal', input_shape=(13,), name="Layer1"),
    Dense(64, activation='relu', kernel_initializer='he_normal', name="Layer2"),
    Dense(128, activation='relu', kernel_initializer='he_normal', name="Layer3"),
    Dense(1, activation='sigmoid', name="Output"),
], name="Model-V1")

# Compile
model1.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Callbacks
cbs = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint("Dense-01.h5", save_best_only=True)
]

In [12]:
model1.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=10,
    callbacks=cbs
)

Epoch 1/10
1253/1253 [==============================] - 4s 3ms/step - loss: 0.0358 - accuracy: 0.9876 - val_loss: 0.0043 - val_accuracy: 0.9993
Epoch 2/10
1253/1253 [==============================] - 3s 3ms/step - loss: 0.0030 - accuracy: 0.9995 - val_loss: 0.0022 - val_accuracy: 0.9994
Epoch 3/10
1253/1253 [==============================] - 3s 3ms/step - loss: 0.0028 - accuracy: 0.9996 - val_loss: 0.0020 - val_accuracy: 0.9995
Epoch 4/10
1253/1253 [==============================] - 3s 2ms/step - loss: 0.0018 - accuracy: 0.9996 - val_loss: 0.0024 - val_accuracy: 0.9995
Epoch 5/10
1253/1253 [==============================] - 3s 3ms/step - loss: 0.0026 - accuracy: 0.9994 - val_loss: 0.0020 - val_accuracy: 0.9996
Epoch 6/10
1253/1253 [==============================] - 3s 3ms/step - loss: 0.0022 - accuracy: 0.9996 - val_loss: 0.0022 - val_accuracy: 0.9995


In [13]:
y_true, y_pred = y_test, np.round(model1.predict(X_test))


392/392 [==============================] - 1s 1ms/step


In [14]:
# Calculate
f1 = f1_score(y_true, y_pred)
acc = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)

# Print
print(f"F1 Score : {f1}\n")
print(f"Accuracy : {acc}\n")
print(f"Precision : {precision}\n")
print(f"Recall : {recall}\n")
print(f"Confusion Matrix : \n{cm}\n")


F1 Score : 0.9997765363128492

Accuracy : 0.9996806642184257

Precision : 0.9998882556710247

Recall : 0.9996648419171043

Confusion Matrix : 
[[3574    1]
 [   3 8948]]

